<a href="https://colab.research.google.com/github/ElenaGordienko1/NLP/blob/main/NLP2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Выпонение лабораторной работы 2

Подготовка, импорт библиотек ⭐

In [73]:
!pip install natasha


In [74]:
import re
from collections import Counter
import math
from natasha import MorphVocab, Segmenter, NewsEmbedding, NewsMorphTagger, Doc

In [75]:
segmenter = Segmenter()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
morph_vocab = MorphVocab()

STOP_WORDS = {"и", "в", "во", "не", "что", "он", "на", "я", "с", "со", "как", "а", "то", "все", "она"}

### Функция лемматизации, удаления стоп-слов и токенизации

In [76]:
def preprocess_text(text: str) -> list[str]:

    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)

    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)

    lemmatized_words = []

    for token in doc.tokens:
        token.lemmatize(morph_vocab)
        lemma = token.lemma
        if lemma is None:
            continue
        if lemma not in STOP_WORDS:
            lemmatized_words.append(lemma)

    return lemmatized_words

### Функция словаря всех уникальных слов

In [77]:
def make_dict(texts: list[list[str]]) -> set[str]:

    all_words = set()
    for text in texts:
        all_words.update(text)
    return all_words


### Bag of Words

In [78]:
def make_bow(texts: list[list[str]]) -> dict[str, int]:

    bow = Counter()
    for text in texts:
        bow.update(text)
    return dict(bow)


### TF-IDF

In [79]:
def compute_tf(text: list[str]) -> dict[str, float]:

    term_count = Counter(text)
    total_terms = len(text)
    return {word: count / total_terms for word, count in term_count.items()}

In [80]:
def compute_idf(texts: list[list[str]]) -> dict[str, float]:

    num_docs = len(texts)
    idf = {}
    all_words = set(word for text in texts for word in text)

    for word in all_words:
        containing_docs = sum(1 for text in texts if word in text)
        idf[word] = math.log(num_docs / (1 + containing_docs)) + 1

    return idf

In [81]:
def compute_tf_idf(texts: list[list[str]]) -> list[dict[str, float]]:

    idf = compute_idf(texts)
    tf_idf_values = []

    for text in texts:
        tf = compute_tf(text)
        tf_idf = {word: tf[word] * idf[word] for word in text}
        tf_idf_values.append(tf_idf)

    return tf_idf_values

### Вывод

In [82]:
texts = [
    "Церкви на Руси строили из дерева и камня. Купола покрывали золотом. Внутри были иконы, фрески, свечи. Монахи переписывали книги, обучали людей вере.",
    "Храмы возводили с высокими сводами и узкими окнами. Колокола звонили к молитве. Фрески изображали библейские сцены. Иконостас отделял алтарь от прихожан. Верующие приносили свечи.",
    "Церкви объединяли народ, учили морали. Священники крестили детей, венчали семьи. Богослужения проходили по праздникам. Монастыри хранили книги, лечили больных. Люди молились, искали покоя."
]

processed_texts = [preprocess_text(text) for text in texts]
dictionary = make_dict(processed_texts)
bow = make_bow(processed_texts)
tf_idf = compute_tf_idf(processed_texts)

print("Словарь всех уникальных слов:", dictionary)
print("Bag of Words:", bow)
print("TF-IDF:", tf_idf)


Словарь всех уникальных слов: {'проходить', 'богослужение', 'окно', 'строить', 'камень', 'сцена', 'приносить', 'больной', 'икона', 'искать', 'мораль', 'купол', 'народ', 'свод', 'венчали', 'звонить', 'семья', 'изображать', 'обучать', 'хранить', 'монах', 'церковь', 'высокий', 'монастырь', 'от', 'отделять', 'библейский', 'священник', 'по', 'из', 'алтарь', 'прихожанин', 'храм', 'русь', 'дерево', 'веровать', 'возводить', 'молиться', 'узкий', 'свеча', 'покой', 'объединять', 'ребенок', 'иконостас', 'молитва', 'колокол', 'книга', 'учить', 'переписывать', 'человек', 'внутри', 'вера', 'крестить', 'быть', 'праздник', 'лечить', 'к', 'золото', 'фреска', 'покрывать'}
Bag of Words: {'церковь': 2, 'русь': 1, 'строить': 1, 'из': 1, 'дерево': 1, 'камень': 1, 'купол': 1, 'покрывать': 1, 'золото': 1, 'внутри': 1, 'быть': 1, 'икона': 1, 'фреска': 2, 'свеча': 2, 'монах': 1, 'переписывать': 1, 'книга': 2, 'обучать': 1, 'человек': 2, 'вера': 1, 'храм': 1, 'возводить': 1, 'высокий': 1, 'свод': 1, 'узкий': 1, '